# Omero File Browser

Browse and download files from an OMERO-image database

## Connecting to OMERO

In [1]:
import ipywidgets as widgets
hosts = ["workshop.openmicroscopy.org", "omero.mri.cnrs.fr"]
port = 4064
CD_host = widgets.Dropdown(
       options=hosts,
       value=hosts[0],
       description='host:')
CD_port = widgets.IntText(
    value=port,
    description='port:',
) 
CD_top = widgets.HBox([CD_host, CD_port])
CD_user = widgets.Text(
    value = 'user',
    description = 'user:'
)
CD_password = widgets.Password(
    value = '',
    description = 'password:'
)
CD = widgets.VBox([CD_top, CD_user, CD_password])
display(CD)

In [105]:
from omero.gateway import BlitzGateway
conn = BlitzGateway(CD_user.value,
                    CD_password.value,
                    host=CD_host.value, port=CD_port.value)
conn.connect()

True

In [131]:
userID = conn.getUser().getId()
groupID = conn.getEventContext().groupId
projects = list(conn.getObjects("Project",
                               opts={'owner': userID,
                                     'group': groupID,
                                     'order_by': 'lower(obj.name)'}))
projectNames = [project.getName() for project in projects]
projectIDs = [project.getId() for project in projects]
projectTupels = list(zip(projectNames, projectIDs))
datasetNames = list([dataset.getName() for dataset in projects[0].listChildren()])
datasetIDs = list([dataset.getId() for dataset in projects[0].listChildren()])
datasetTupels = list(zip(datasetNames, datasetIDs))
BD_project = widgets.Dropdown(
       continuous_update=True,
       options=projectTupels,
       value=projectTupels[0][1],
       description='project:')
BD_dataset = widgets.Dropdown(
       options=datasetTupels,
       value=datasetTupels[0][1],
       description='dataset:')
out = widgets.Output()
BD_top = widgets.HBox([BD_project, BD_dataset, out])
def on_value_change(change):
    datasetNames = list([dataset.getName() for dataset in projects[change['new']].listChildren()])
    datasetIDs = list([dataset.getId() for dataset in projects[change['new']].listChildren()])
    datasetTupels = list(zip(datasetNames, datasetIDs))
    BD_dataset.options = datasetTupels
    BD_dataset.value = datasetTupels[0][1]
BD_project.observe(on_value_change, names="index")
display(BD_top)